In [23]:
import os
from dotenv import load_dotenv
import openai
from langchain_openai import ChatOpenAI
from langchain.prompts import (PromptTemplate, 
                               SystemMessagePromptTemplate, 
                               ChatPromptTemplate,
                              HumanMessagePromptTemplate)
from langchain.llms import OpenAI

In [24]:
# Setup model
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
openai.api_key = api_key
model = ChatOpenAI(openai_api_key=api_key)

In [25]:
# Step 1: import Parser
from langchain.output_parsers import CommaSeparatedListOutputParser
output_parser = CommaSeparatedListOutputParser()
# Step 2 - format instructions
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [26]:
# Test
# reply = "red, blue, green"
# output_parser.parse(reply)

In [27]:
humman_template = "{request}\n{format_instructions}"
humman_prompt = HumanMessagePromptTemplate.from_template(humman_template)

In [28]:
chat_prompt = ChatPromptTemplate.from_messages([humman_prompt])

In [29]:
request = chat_prompt.format_prompt(request="give me 5 characteristics of dogs",
                         format_instructions=output_parser.get_format_instructions())
request

ChatPromptValue(messages=[HumanMessage(content='give me 5 characteristics of dogs\nYour response should be a list of comma separated values, eg: `foo, bar, baz`')])

In [30]:
model_request = request.to_messages()
result = model.invoke(model_request)
result.content

'loyal, playful, social, protective, intelligent'